In [1]:
import torch
import torch.nn as nn
import numpy as np
from data_rnn import load_ndfa, load_brackets
from utils import device

In [2]:
x_train, (i2w, w2i) = load_ndfa(n=150_000)

In [37]:
class DataLoader():
    def __init__(self, x, maxsize=10000):
        self.x = [[w2i['.start']] + i + [w2i['.end']] for i in x]
        self.c = 0
        self.maxsize = max(maxsize, len(max(self.x, key=lambda x: len(x))) + 1)
    
    def shuffle(self):
        self.c = 0
        self.x = [self.x[i] for i in np.random.permutation(range(len(self.x)))]
    
    def get(self):
        temp = []
        while len(temp) < self.maxsize:
            if self.c >= len(self.x) or len(temp) + len(self.x[self.c]) + 1 > self.maxsize:
                temp.extend([w2i['.pad']] * (self.maxsize - len(temp)))
            else:
                temp.extend(self.x[self.c] + [w2i['.pad']])
                self.c += 1
        return torch.tensor(temp, dtype=torch.long)
    
    def dataloader(self, bs):
        while self.c < len(self.x):
            x = torch.concat([self.get().view(1, -1) for _ in range(bs)]).to(device)
            x = x[x.count_nonzero(dim=1) > 1]
            y = torch.zeros_like(x)
            y[:, :-1] = x[:, 1:]
            yield x, y

In [40]:
class Network(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.emb = nn.Embedding(len(i2w), 32)
        self.lstm = nn.LSTM(32, 16, batch_first=True)
        self.linear = nn.Linear(16, len(w2i))
    
    def forward(self, x):
        x = self.emb(x)
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x

In [41]:
model = Network().to(device)
dl = DataLoader(x_train)

In [42]:
model(next(iter(dl.dataloader(32)))[0]).shape

torch.Size([32, 10000, 15])

In [1]:
x_train, (i2w, w2i) = load_brackets(n=150_000)

NameError: name 'load_brackets' is not defined